In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
# This should be the only variable to change per corpus
NAME = 'cardib'

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

PATH = '/content/drive/My Drive/LA Hacks/' 
DICT_PATH = PATH + 'Dictionaries/' + 'char_index_' + NAME + '.p'
MODEL_PATH = PATH + 'Model Weights/' + NAME + '.h5'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Load corpus
path = PATH + 'cardib_corpus.txt'
with open(path, encoding='unicode_escape') as f:
    text = f.read().lower()

In [7]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 63


In [0]:
import pickle

with open(DICT_PATH, 'wb') as f:
  pickle.dump(char_indices, f)

In [9]:
print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~']


In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 82588


In [11]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [12]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
# Load model
#model.load_weights(MODEL_PATH)

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

checkpoint = ModelCheckpoint(MODEL_PATH, monitor='loss', verbose=0, save_best_only=True, save_weights_only=True)

early_stopping = EarlyStopping(monitor='loss', patience=3)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)


In [0]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[generate_text, checkpoint, early_stopping])

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
82588/82588 [==============================] - 60s 728us/step - loss: 2.2817

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " was not ready for the typa fire cardi w"
 was not ready for the typa fire cardi wann the mang the mars the banger hat and ballang allays all allay @iamcardib when the bat song the baby https://t.co/5x0xqmbj
 @challallanaman: @iamcardib https://t.co/xxlcgh6
 @chellaname: @iamcardib wanne that i lake i lake the mars all allass allay i lake i last mading the ball and @iamcardib  @iamcardib https://t.co/xxxxuxle
 @chanamare: @iamcardib is me the ball allay @iamcardib money by card
----- diversity: 0.5
----- Generating with seed: " was not ready for the typa fire cardi w"
 was not ready for the typa fire cardi wand las bab lole  https://t.co/5xqrjeby
 @schallacamann: @iamcardib https://t.co/060aqezby
 @chichalllealre: @iamcardib is i me the and alleds me song gealled it

In [0]:
!ls

drive  gdrive  nietzsche.h5  sample_data
